Code Introduction:

This is the one of the latest versions of the code, but sadly not exacly the same as the one reached the max score (the score was similar through most of my submissions)

As the dataset is based off texts, similarily to the dataset used in the lab itself, I used a lot of the same code to create the model.
Tried both CountVectorizer and TF/IDF one (ended up using the cound, as it got better results), also tried different sizes of BagOfWords/tokenizers, also different models (DFTree/Multi_nb/Keras etc) and layer sizes. This is the final configulation attemped with this specific model.

In [1]:
# Default code importing Kaggle files + getting the paths to the files

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Added this package to clean the text
!pip install cleantext

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dm-lab2/tweets_DM.json
/kaggle/input/dm-lab2/sampleSubmission.csv
/kaggle/input/dm-lab2/data_identification.csv
/kaggle/input/dm-lab2/emotion.csv


In [2]:
# Part 1-

# imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import nltk
import keras
from keras.callbacks import CSVLogger
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from cleantext import clean
import warnings
import json
import time

warnings.filterwarnings('ignore')

#imports + readFiles

KAGGLE_JSON = '/kaggle/input/dm-lab2/tweets_DM.json'
KAGGLE_DIVIDE = '/kaggle/input/dm-lab2/data_identification.csv'
KAGGLE_EMOTIONS = '/kaggle/input/dm-lab2/emotion.csv'

LOGS_FILE = '/kaggle/working/training_log.csv'
 
data_from_json = pd.read_json(KAGGLE_JSON, lines=True, orient='records')

data_division = pd.read_csv(KAGGLE_DIVIDE)
data_emotions = pd.read_csv(KAGGLE_EMOTIONS)

print("data.shape", data_from_json.shape)
print("data_division.shape", data_division.shape)
print("data_emotions.shape", data_emotions.shape)

data_from_json.head()


data.shape (1867535, 5)
data_division.shape (1867535, 2)
data_emotions.shape (1455563, 2)


,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [3]:
# creating different DataFrame to hold the text itself from the lower level in the dict 
data_internal = pd.DataFrame(data_from_json["_source"].apply(lambda x: x["tweet"]).to_list())

# cleaning stage 1- replace all of the hashtags with the actual words (this way the hashtag sign won't intervene with the text analysis)
def replace_all(text, hasht):
    for h in hasht:
        text = text.replace(f'#{h}', h)
    return text
data_internal["text_combined"] = data_internal.apply(lambda x: replace_all(x["text"], x["hashtags"]), axis=1)
data_internal.head()

,hashtags,tweet_id,text,text_combined
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...","People who post ""add me on Snapchat"" must be d..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...","@brianklaas As we see, Trump is dangerous to f..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...","Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...","""Trust is not the same as faith. A friend is s..."


In [4]:
start_time = time.time()

# cleaning stage 2- remove other special signs/sunctuations/numbers and lowercase the text as a whole to have a unified structure
data_internal['text_combined'] = data_internal['text_combined'].apply(lambda x: clean(x, lowercase=True, numbers=True, punct=True))
print("finished clean time:", time.time() - start_time)
data_internal.head()

finished clean time: 525.4585771560669


,hashtags,tweet_id,text,text_combined
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",people who post add me on snapchat must be deh...
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",brianklaas as we see trump is dangerous to fre...
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",confident of your obedience i write to you kno...
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,now issa is stalking tasha 😂😂😂 lh
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",trust is not the same as faith a friend is som...


In [5]:
#check if the tweet_ids order don't match between the different files. if this is really the case, then set all ids to be tweet_id
check_orders=pd.DataFrame({"internal_id": data_internal["tweet_id"], "division_id": data_division["tweet_id"]})
if len(check_orders[check_orders["internal_id"] != check_orders["division_id"]]) > 0:
    data_internal.set_index('tweet_id', inplace = True, drop = False)
    data_division.set_index('tweet_id', inplace = True, drop = False)
    data_emotions.set_index('tweet_id', inplace = True, drop = False)
    
#check again if ids match (should be empty)
check_orders2=pd.DataFrame({"internal_id": data_internal["tweet_id"], "division_id": data_division["tweet_id"]})
check_orders2[check_orders2["internal_id"] != check_orders2["division_id"]]

,internal_id,division_id
tweet_id,,


In [7]:
# divide the data into train and test (which has no knwon emotions) then add the emotions to the train set
print("data_division.shape", data_division.shape)

data_internal["class"] = data_division["identification"]

print("data_emotions.shape", data_emotions.shape)
test_lines = data_internal[data_internal["class"] == "test"]
train_lines = data_internal[data_internal["class"] == "train"]

train_lines["emotion"] = data_emotions["emotion"]

print("all emotions merged:", train_lines["emotion"].unique())

data_division.shape (1867535, 2)
data_emotions.shape (1455563, 2)
all emotions merged: ['anticipation' 'sadness' 'fear' 'joy' 'anger' 'trust' 'disgust'
 'surprise']


In [9]:
# saparating the data to train+test to have a supervised testset as well (10% of the train dataset)
split_X_train, split_X_test, y_train, y_test = train_test_split(train_lines['text_combined'], train_lines['emotion'], test_size=0.2)

#make sure the ratios are okay with the smaller test size, so the data can be learned well across all emotions.
count = pd.DataFrame({"train": y_train.value_counts(), "test": y_test.value_counts()})
count["ratio"] = count.apply(lambda x: x['test']/x['train'], axis=1) #ideally: all ratios are of the same area
count["train_ratio"] = count.apply(lambda x: x['train']/len(y_train), axis=1)
count["test_ratio"] = count.apply(lambda x: x['test']/len(y_test), axis=1) # ideally: train ratio and test ratio are about the same
count

,train,test,ratio,train_ratio,test_ratio
emotion,,,,,
joy,412724,103293,0.250271,0.354437,0.354821
anticipation,199016,49919,0.250829,0.170910,0.171476
trust,164429,41049,0.249646,0.141207,0.141007
sadness,154629,38808,0.250975,0.132791,0.133309
disgust,111538,27563,0.247118,0.095786,0.094681
fear,51112,12887,0.252133,0.043894,0.044268
surprise,39057,9672,0.247638,0.033541,0.033224
anger,31945,7922,0.247989,0.027434,0.027213


In [11]:
start_time = time.time()

# build BOW with quite large size (to support as many keywords as possible) and remove all stopwords, then apply it it to trainset
BOW_1500 = CountVectorizer(max_features=1500, stop_words='english', tokenizer=nltk.word_tokenize) 

BOW_1500.fit(split_X_train)

X_train = BOW_1500.transform(split_X_train)
X_test = BOW_1500.transform(split_X_test)

print("BOW_1500 creation time:", time.time() - start_time)
X_train

BOW_1500 creation time: 635.5589420795441


<1164450x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 11637701 stored elements in Compressed Sparse Row format>

In [15]:
# create Keras model (+encoder and decoder) similarily to how it was made in the lab, with new layersizes- we now have 8 output options and the folloeing layers are size 8*4 and 8*8 accordingly
start_time = time.time()
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train_enc = label_encode(label_encoder, y_train)
y_test_enc = label_encode(label_encoder, y_test)

input_shape = X_train.shape[1]
output_shape = len(label_encoder.classes_)
print(len(label_encoder.classes_), " CLASSES: ", label_encoder.classes_)
print("input_shape: ", input_shape, "; output_shape: ", output_shape)

# input layer
model_input = Input(shape=(input_shape, ))
X = model_input

# 1st hidden layer
X_W1 = Dense(units=128)(X)  # 128
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=32)(H1)  # 32
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 8
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()
logs_file = LOGS_FILE

csv_logger = CSVLogger(logs_file)
print("Keras model creation time:", time.time() - start_time)

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(X_train, y_train_enc, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test_enc))
print('training finish')

training_log = pd.DataFrame()
training_log = pd.read_csv(logs_file)

pred_result = model.predict(X_test, batch_size=128)
pred_result = label_decode(label_encoder, pred_result)

print('keras testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test_enc), pred_result), 2)))
print("Keras model fit time (from start):", time.time() - start_time)
pred_result

8  CLASSES:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']
input_shape:  1500 ; output_shape:  8
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1500)]            0         
                                                                 
 dense (Dense)               (None, 128)               192128    
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 1032     

array(['anticipation', 'joy', 'joy', ..., 'joy', 'joy', 'joy'],
      dtype=object)

In [16]:
# save results summary into json
res_summary = {
    "keras":{
        "test": format(round(accuracy_score(label_decode(label_encoder, y_test_enc), pred_result), 2))
    }
}
with open("summary.json", "w") as outfile:
    json.dump(res_summary, outfile)

# save all supervised test results into 1 df and export as CSV
# all_test_responses = pd.DataFrame({"id": test_sup['tweet_id'], "DT": y_test_pred, "mnb": y_test_pred_mnb, "ker": pred_result})
all_test_responses = pd.DataFrame({"id": split_X_test, "emotion": pred_result})
all_test_responses.to_csv('test_sup_res.csv')

In [17]:
# run against real test dataset (and not the supervised one):
X_test_real = BOW_1500.transform(test_lines['text_combined'])
y_test_pred_ker_enc = model.predict(X_test_real, batch_size=128)
y_test_pred_ker = label_decode(label_encoder, y_test_pred_ker_enc)

#save submission file
submit_responses_mnb = pd.DataFrame({"id": test_lines['tweet_id'], "emotion": y_test_pred_ker})
submit_responses_mnb.set_index("id").to_csv('submission_ker.csv')

3219/3219 [==============================] - 10s 3ms/step
